In [19]:
# This file aims at converting units that are not grammes in grammes
# we consider here that : 1 c.à.c = 5 g and 1 c.à.s = 15 grammes

In [1]:
!pip install lxml 

In [4]:
from scrapping_marmiton import *
import re


In [7]:
corres = open("Poids.txt", "r")
poids=corres.readlines()
dicopoids={}
for ligne in poids:
    dicopoids[str(''.join(z for z in ligne if not z.isdigit()).replace(" \n", "").lower())]=int("".join(re.findall('\d',str(ligne))))
     

In [5]:
dicopoids

{'ail tête': 80,
 'ail gousse': 7,
 'artichaut camus': 350,
 'artichaut castel': 500,
 'artichaut poivrade': 60,
 'artichaut violet': 150,
 'asperges': 50,
 'aubergine': 400,
 'carotte': 125,
 'céleri-rave': 1000,
 'chou-fleur': 1500,
 'chou vert': 2000,
 'concombre': 200,
 'courgette': 200,
 'cresson': 200,
 'échalote': 40,
 'laitue ,\n': 250,
 'navet': 250,
 'oignon': 100,
 'poireau': 150,
 'poivron': 300,
 'pomme de terre': 120,
 'tomate': 125,
 'abricot': 55,
 'ananas': 2000,
 'banane': 150,
 'citron': 110,
 'clémentine': 70,
 'datte': 35,
 'figue noire': 50,
 'melon': 650,
 'orange': 200,
 'pamplemousse': 400,
 'pêche': 150,
 'poire': 120,
 'pomme': 175,
 'olive': 4,
 'câpre .\n': 5,
 'feuille .\n': 5,
 'sel': 1,
 'poivre': 1,
 'clou': 1,
 'jambon': 65,
 'steak': 200,
 'pruneau': 15,
 'piment': 30,
 'cerise': 6,
 'fraise': 10,
 'avocat': 220,
 'noix': 8}

In [26]:
url_test = "https://www.marmiton.org/recettes/recette_moelleux-au-chocolat-sans-beurre-sans-sucre_14748.aspx"

test=find_all_recipes("steak tartare")



  0%|          | 0/15 [00:00<?, ?it/s]

In [27]:
test

{'Steak tartare': {'lien': 'https://www.marmiton.org/recettes/recette_steak-tartare_18121.aspx',
  'recette': {'moutarde de Dijon': '1c.à.c',
   'oignon': '1c.à.s',
   'câpres': '1c.à.s',
   'sauce worcestershire': '1c.à.c',
   'ketchup': '1c.à.s',
   'tabasco': '',
   'poivre': '',
   'sel': '',
   "huile d'olive": '2c.à.s',
   'persil': '1c.à.c',
   'boeuf': '250g',
   "jaune d'oeuf": '1'},
  'nombre de personnes': 2.0},
 'Steak tartare arrangé': {'lien': 'https://www.marmiton.org/recettes/recette_steak-tartare-arrange_50144.aspx',
  'recette': {'boeuf haché': '450g',
   'huile': '',
   'moutarde': '',
   'ketchup': '',
   'tabasco': '',
   'poivre': '',
   'sel': '',
   'câpres': '',
   'persil': '',
   'oeuf': '1',
   'échalote': '1',
   'cornichons': '',
   'ciboulette': ''},
  'nombre de personnes': 2.0},
 'Steak tartare végétarien': {'lien': 'https://www.marmiton.org/recettes/recette_steak-tartare-vegetarien_30154.aspx',
  'recette': {'carotte': '500g',
   'câpres': '2c.à.s',
  

In [10]:
float(("".join(re.findall('\d',str(0.25)))))

25.0

In [ ]:
def convert(dico):
    for x,y in dico.items():
        
            
                for k,v in y['recette'].items():
            
                    if v != '':
                        if '⁄' in v:
                            if '1⁄4' in v:
                                n=0.25
                            if '1⁄3' in v:
                                n=0.33
                            if '1⁄2' in v:
                                n=0.5
                            for a,b in dicopoids.items():
                                if a in k:
                                    y['recette'].update({k:str(b*n)+"g"})
                
                        else:
                            n=float(("".join(re.findall('\d',str(v)))))
                
                            l=str(''.join(z for z in v if not z.isdigit()))
                
                
                
                
                            if l=='c.à.s rase':
                                y['recette'].update({k:str(15*n)+"g"})
                            elif k=="jaune d'oeuf":
                                y['recette'].update({k:str(20*n)+"g"})
                            elif k=="oeuf":
                                y['recette'].update({k:str(55*n)+"g"})
                            elif l=='c.à.c':
                                y['recette'].update({k:str(5*n)+"g"})
                            elif l=='tranche':
                                y['recette'].update({k:str(32*n)+"g"})
                            elif 'brin' or 'trait' in l:
                                y['recette'].update({k:str(5*n)+"g"})
                            elif l=='ml':
                                y['recette'].update({k:str(1*n)+"g"})
                            elif l=='cl':
                                y['recette'].update({k:str(10*n)+"g"})
                            elif l=='dl':
                                y['recette'].update({k:str(100*n)+"g"}) 
                            elif l=='l':
                                y['recette'].update({k:str(1000*n)+"g"})    
                            elif l=='c.à.s':
                                y['recette'].update({k:str(15*n)+"g"})
                            elif l=='cuillères':
                                y['recette'].update({k:str(15*n)+"g"})
                            if l!='g':    
                                for a,b in dicopoids.items():
                                    if a in k:
                                        y['recette'].update({k:str(b*n)+"g"})
                    elif v=="": #on définit une valeur standardisé pour les éléments manquants. Ces élements sont souvent utilisés en des quantités limitées.
                        y['recette'].update({k:str(2)+"g"})
                
    
                    
                    
            
            
                
            
            

In [46]:
def essai(dico):
    for x,y in dico.items():
        
            
                for k,v in y['recette'].items():
                    print(k,v)

In [47]:
essai(test)

moutarde de Dijon 1c.à.c
oignon 1c.à.s
câpres 1c.à.s
sauce worcestershire 1c.à.c
ketchup 1c.à.s
tabasco 
poivre 
sel 
huile d'olive 2c.à.s
persil 1c.à.c
boeuf 250g
jaune d'oeuf 1
boeuf haché 450g
huile 
moutarde 
ketchup 
tabasco 
poivre 
sel 
câpres 
persil 
oeuf 1
échalote 1
cornichons 
ciboulette 
carotte 500g
câpres 2c.à.s
poivron rouge 1
oignon 1
persil 1c.à.s
moutarde 2c.à.s
tabasco 1trait
poivre 
sel 
mayonnaise 3c.à.s
sauce anglaise 1c.à.s
moutarde de Dijon 1c.à.s
fromage blanc 6cuillères
poivre 
sel 
câpres 1c.à.c
ciboulette 
échalote 1
mayonnaise 
filet de boeuf 250g
huile d'olive 2c.à.s
moutarde 1c.à.s
ketchup 1c.à.s
câpres 20
olives 7
tomates séchées 4
Parmesan 50g
sel 
poivre 
tabasco 
jaune d'oeuf 1
oignons nouveaux 2
mozzarella 1
ciboulette 2c.à.s
boeuf haché 300g
moutarde 1c.à.c
ketchup 
poivre 
sel 
jaunes d'oeuf 2
échalotes 2
viande hachée 150g
sésame 1c.à.c
huile de sésame 1c.à.c
pomme 1⁄4
ail 1petite gousse
jus de citron 
poire 1⁄4
oeuf 1
poivron rouge 1
persil 
mou

In [40]:
convert(test)

In [41]:
test

{'Steak tartare': {'lien': 'https://www.marmiton.org/recettes/recette_steak-tartare_18121.aspx',
  'recette': {'moutarde de Dijon': '1c.à.c',
   'oignon': '1c.à.s',
   'câpres': '1c.à.s',
   'sauce worcestershire': '1c.à.c',
   'ketchup': '1c.à.s',
   'tabasco': '',
   'poivre': '',
   'sel': '',
   "huile d'olive": '2c.à.s',
   'persil': '1c.à.c',
   'boeuf': '250g',
   "jaune d'oeuf": '1'},
  'nombre de personnes': 2.0,
  'moutarde de Dijon': '5.0g',
  'oignon': '100.0g',
  'câpres': '5.0g',
  'sauce worcestershire': '5.0g',
  'ketchup': '5.0g',
  'tabasco': '2g',
  'poivre': '2g',
  'sel': '2g',
  "huile d'olive": '8.0g',
  'persil': '5.0g',
  'boeuf': '1250.0g',
  "jaune d'oeuf": '20.0g'},
 'Steak tartare arrangé': {'lien': 'https://www.marmiton.org/recettes/recette_steak-tartare-arrange_50144.aspx',
  'recette': {'boeuf haché': '450g',
   'huile': '',
   'moutarde': '',
   'ketchup': '',
   'tabasco': '',
   'poivre': '',
   'sel': '',
   'câpres': '',
   'persil': '',
   'oeuf': 

In [40]:
def keskimank(dico):
    for x,y in dico.items():
        for k,v in y.items():
                l=str(''.join(z for z in v if not z.isdigit()))
                if 'g' not in l:
                    print(k)
                    

In [69]:
keskimank(test)

Saint Morêt
yaourt nature
mangue
mangue
ciboulette
feta
roquette
vinaigre de riz
mangue
cumin
mangue


In [42]:
test

{'Verrine avocat et saumon fumé': {'citron': '1452000.0g',
  'avocat': '484000.0g',
  'truite': '500.0g',
  'fromage frais': '25000.0g',
  'fromage ail et fines herbes': '12500.0g',
  'ciboulette': '10.0g',
  'oeuf de truite': '250.0g'},
 "Terrine de saumon à l'avocat": {'citrons': '242000.0g',
  'gélatine': '2500.0g',
  'crème fraîche': '5000.0g',
  'poivre': '10.0g',
  'sel': '10.0g',
  'saumon frais': '75000.0g',
  'saumon fumé': '1500.0g',
  'avocats': '1452000.0g',
  'court-bouillon': '250.0g'},
 'Verres avocat, saumon, mangue': {'citron': '121000.0g',
  'paprika': '10.0g',
  'saumon fumé': '750.0g',
  'avocats': '968000.0g',
  'mangue': '50000.0g',
  'crème fraîche liquide': '5000.0g',
  'ciboulette': '1250.0g'},
 "Petits verres d'avocat au saumon et à la feta": {'citron': '121000.0g',
  'aneth': '10.0g',
  'avocats': '968000.0g',
  'feta': '12500.0g',
  'saumon fumé': '25000.0g',
  'surimi': '25000.0g'},
 'Verrine 3 couleurs : Avocat, Saumon, Cream Cheese ': {'citron': '1452000.

In [61]:
def usable(dico):
    for x,y in dico.items():
        for k,v in y.items():
                n=int(("".join(re.findall('\d',str(v)))))
                y.update({k:int(n)})

In [9]:
usable(test)
test

{'Steak tartare': {'moutarde de Dijon': '1c.à.c',
  'oignon': '1c.à.s',
  'câpres': '1c.à.s',
  'sauce worcestershire': '1c.à.c',
  'ketchup': '1c.à.s',
  'tabasco': '',
  'poivre': '',
  'sel': '',
  "huile d'olive": '2c.à.s',
  'persil': '1c.à.c',
  'boeuf': '250g',
  "jaune d'oeuf": '1'},
 'Steak tartare arrangé': {'huile': '450g',
  'moutarde': '',
  'ketchup': '',
  'tabasco': '',
  'poivre': '',
  'sel': '',
  'câpres': '',
  'persil': '',
  'boeuf haché': '',
  'oeuf': '1',
  'échalote': '1',
  'cornichons': ''},
 'Steak tartare végétarien': {'carotte': '500g',
  'câpres': '2c.à.s',
  'poivron rouge': '1',
  'oignon': '1',
  'persil': '1c.à.s',
  'moutarde': '2c.à.s',
  'tabasco': '1trait',
  'poivre': '',
  'sel': '',
  'mayonnaise': '3c.à.s',
  'sauce anglaise': '1c.à.s'},
 'Sauce légère et relevée pour steak tartare': {'moutarde de Dijon': '1c.à.s',
  'fromage blanc': '6cuillères',
  'poivre': '',
  'sel': '',
  'câpres': '1c.à.c',
  'ciboulette': '',
  'échalote': '1',
  'ma